In [7]:
import requests
import csv
import json
import re
import ast

In [8]:
url = 'https://nookplaza.net/main.dart.js?v=7'
r = requests.get(url)
mainJs = r.text

In [9]:
# 'C.Iz=new H.b(13,{},C.a,[P.a,P.a])'
m = re.findall('new H\.b\(13,(.*),C\.a,\[P\.a,P\.a\]\)', mainJs)
print(len(m))

m = [item.encode("utf-8").decode('unicode-escape') for item in m]
m = [ast.literal_eval(item) for item in m if not 'null' in item]
print(len(m))

nameMap = {}
for item in m:
    nameMap[item["en-us"].lower()] = item

5715
5713


In [10]:
nameMap['street organ']

{'de-de': 'Drehorgel',
 'en-gb': 'street organ',
 'en-us': 'street organ',
 'es-es': 'organillo en carro',
 'fr-ca': 'orgue de barbarie',
 'fr-fr': 'orgue de barbarie',
 'it-it': 'organo a rullo',
 'ja-jp': 'てまわしオルガン',
 'ko-kr': '배럴 오르간',
 'nl-nl': 'draaiorgel',
 'ru-ru': 'шарманка',
 'zh-cn': '手摇风琴',
 'zh-tw': '手搖風琴'}

In [11]:
url = 'https://api.nookplaza.net/items?category=Furniture'
r = requests.get(url)
data = r.json()['results']

In [42]:
idMap = {}
bodySet = set()
patternSet = set()
output = [['id', '名稱(英文)', '名稱(中文)', '買價', '賣價', '分類', '標籤', '主題', '可互動', '大小', '取得方式', 'DIY', '圖片', '款式說明', '樣式說明', '款式改造', '樣式改造', '款式', '樣式', '檔名對照索引']]
for item in data:
    if item["category"] == "Art":
        continue
    name = nameMap[item['name'].lower()]
    nameMap[item['name'].lower()]['id'] = item['content']['internalID']
    idMap[item['content']['internalID']] = item
    
    # 名稱 買價 賣價
    row = [int(item['content']['internalID']), name['en-us'], name['zh-tw'], item['content']['buy'], item['content']['sell']]

    # 分類 標籤 主題
    # 'category' 'tag' 'themes'
    if "None" in item['content']['themes']:
        item['content']['themes'].remove("None")
    row += [item['content']['category'], item['content']['tag'], '、'.join(item['content']['themes'])]
    
    # 互動 大小
    row += [item['content']['interact'], '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])]

    # 取得方式 DIY
    if not 'dIY' in item['content']:
        print(item)
    row += ['、'.join(item['content']['obtainedFrom']), item['content']['dIY']]

    # 圖片檔名
    row += [item['content']['filename']]
    
    # 改造-款式 樣式
    # bodyCustomize bodyColor bodyTitle; patternCustomize patternColor patternTitle
    row += [item['content']['bodyTitle'], item['content']['patternTitle'], item['content']['bodyCustomize'], item['content']['patternCustomize']]

    variations = {
        "bodys": {},
        "pattrens": {}
    }
    for variation in item['variations']:
        content = variation['content']
        filename = content['filename']
        bodyIndex, pattrenIndex = filename.split('_')[-2:]
        
        bodyColor = '' if content['bodyColor'] is None else content['bodyColor']
        patternColor = '' if content['patternColor'] is None else content['patternColor']
        
        variations['bodys'][bodyColor] = bodyIndex
        variations['pattrens'][patternColor] = pattrenIndex
        
        bodySet.add(bodyColor)
        patternSet.add(patternColor)

    row += ['、'.join(list(variations['bodys'].keys())), '、'.join(list(variations['pattrens'].keys())), variations]
       
#     print(row)
    output.append(row)
output[1:] = sorted(output[1:], key=lambda k: k[0])

In [43]:
with open('items.csv', 'wt', encoding='utf-8', newline='') as fout:
    csvout = csv.writer(fout)
    csvout.writerows(output)

# for pattern in patternSet:
#     print(pattern)